<a href="https://colab.research.google.com/github/ArtemJDS/NN-for-population-approximation/blob/main/LSTM_for_population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import h5py
import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from google.colab import files
import seaborn as sns
from sklearn.preprocessing import normalize
import pandas as pd
from scipy.special import softmax
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import TimeseriesGenerator
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/voltage_and_input.zip

In [ ]:
N_BINS = 249
INPUT_EPOCH = 2500

In [ ]:
dataset = h5py.File('/content/content/voltage_and_input.h5', 'r')
voltage = np.array(dataset['voltage'], dtype = np.float32).T    # population's voltage dynamics               
input = np.array(dataset['input'], dtype = np.float32)     # population's input   

min = np.min(voltage)
max = np.max(voltage)
bins = np.linspace(min, max, N_BINS)     
x = np.apply_along_axis(np.histogram, 1, voltage, bins = bins, density = True).T[0]
x = list(x)
x = np.stack(x)
densities = normalize(x, norm = 'l1') # shape = n,m, where n = # of timesteps, m = N_BINS - 1

In [ ]:
LENGTH = 1
data_gen = TimeseriesGenerator(densities, densities,
                               length=length, sampling_rate=1,
                               batch_size=128, shuffle=True)

Approximating of dynamics via LSTM network

In [ ]:
model = Sequential()
#model.add(Dense(n_bins, activation='sigmoid', name = 'initial_layer'))
model.add(LSTM(256, activation='sigmoid',
               input_shape=(LENGTH, N_BINS),
               return_sequences=True, 
               ))
model.add(LSTM(256, activation='sigmoid', return_sequences=True))
model.add(LSTM(256, activation='sigmoid'))
model.add(Dense(N_BINS, activation='softmax'))
model.compile(optimizer='adam', loss= "mse")
model.summary()
model.fit(data_gen, epochs=200)